# Task 1: Pulling Data from Ken French Website using `pandas_datareader`

In [1]:
from pandas_datareader.famafrench import get_available_datasets
import pandas_datareader.data as web
import sys
sys.path.insert(0, '../src')
from pull_raw_data import *
from pull_test_data import *

We list the portfolios we seek to replicate, and build a function to systematically read in each of the testing datasets required to compare our replication results

Each portfolio is constructed from different dates, so a parameter in our function is start/ end dates for what we should read in.

In [2]:
ports = list(portfolio_descriptions.keys())
filepath = f'../data/famafrench/{ports[0]}.xlsx'
descrip = pd.read_excel(filepath, sheet_name= 'Description')
print(descrip['Description'][0])

6 Portfolios ME EP 2x3
----------------------

This file was created by CMPT_ME_BEME_RETS using the 202312 CRSP database. It contains value- and equal-weighted returns for the intersections of 2 ME portfolios and 3 E/P portfolios. The portfolios are constructed at the end of Jun. ME is market cap at the end of Jun. E/P is is earnings before extraordinary at the last fiscal year end of the prior calendar year divided by ME at the end of December of the prior year. Firms with negative or zero E/P are not included in any portfolio. The annual returns are from January to December. Missing data are indicated by -99.99 or -999. The break points include utilities and include financials. The portfolios include utilities and include financials. Copyright 2023 Kenneth R. French

  0 : Value Weight Average Returns -- Monthly (870 rows x 6 cols)
  1 : Equal Weight Average Returns -- Monthly (870 rows x 6 cols)
  2 : Value Weight Average Returns -- Annual (72 rows x 6 cols)
  3 : Equal Weight Avera

In [3]:
filepath = f'../data/famafrench/{ports[2]}.xlsx'
descrip = pd.read_excel(filepath, sheet_name= 'Description')
print(descrip['Description'][0])

6 Portfolios ME CFP 2x3
-----------------------

This file was created by CMPT_ME_BEME_RETS using the 202312 CRSP database. It contains value- and equal-weighted returns for the intersections of 2 ME portfolios and 3 CF/P portfolios. The portfolios are constructed at the end of Jun. ME is market cap at the end of Jun. CF/P is cashflow (earnings before extraordinary plus deferred tax income statement plus common equitys share of depreciation) at the last fiscal year end of the prior calendar year divided by ME at the end of December of the prior year. Firms with negative or zero CF/P are not included in any portfolio. The annual returns are from January to December. Missing data are indicated by -99.99 or -999. The break points include utilities and include financials. The portfolios include utilities and include financials. Copyright 2023 Kenneth R. French

  0 : Value Weight Average Returns -- Monthly (870 rows x 6 cols)
  1 : Equal Weight Average Returns -- Monthly (870 rows x 6 cols

In [4]:
filepath = f'../data/famafrench/{ports[4]}.xlsx'
descrip = pd.read_excel(filepath, sheet_name= 'Description')
print(descrip['Description'][0])

6 Portfolios ME DP 2x3
----------------------

This file was created by CMPT_ME_BEME_RETS using the 202312 CRSP database. It contains value- and equal-weighted returns for the intersections of 2 ME portfolios and 3 D/P portfolios. The portfolios are constructed at the end of Jun. ME is market cap at the end of Jun. D/P is the total dividends paid from July of t-1 to June of t per dollar of equity in June of t. Firms with negative or zero D/P are not included in any portfolio. The annual returns are from January to December. Missing data are indicated by -99.99 or -999. The break points include utilities and include financials. The portfolios include utilities and include financials. Copyright 2023 Kenneth R. French

  0 : Value Weight Average Returns -- Monthly (1158 rows x 6 cols)
  1 : Equal Weight Average Returns -- Monthly (1158 rows x 6 cols)
  2 : Value Weight Average Returns -- Annual (96 rows x 6 cols)
  3 : Equal Weight Average Returns -- Annual (96 rows x 6 cols)
  4 : Number

In [5]:
filepath = f'../data/famafrench/{ports[6]}.xlsx'
descrip = pd.read_excel(filepath, sheet_name= 'Description')
print(descrip['Description'][0])

5 Industry Portfolios
---------------------

This file was created by CMPT_IND_RETS using the 202312 CRSP database. It contains value- and equal-weighted returns for 5 industry portfolios. The portfolios are constructed at the end of June. The annual returns are from January to December. Missing data are indicated by -99.99 or -999. Copyright 2023 Kenneth R. French

  0 : Average Value Weighted Returns -- Monthly (1170 rows x 5 cols)
  1 : Average Equal Weighted Returns -- Monthly (1170 rows x 5 cols)
  2 : Average Value Weighted Returns -- Annual (97 rows x 5 cols)
  3 : Average Equal Weighted Returns -- Annual (97 rows x 5 cols)
  4 : Number of Firms in Portfolios (1170 rows x 5 cols)
  5 : Average Firm Size (1170 rows x 5 cols)
  6 : Sum of BE / Sum of ME (98 rows x 5 cols)
  7 : Value-Weighted Average of BE/ME (98 rows x 5 cols)


In [6]:
filepath = f'../data/famafrench/{ports[9]}.xlsx'
descrip = pd.read_excel(filepath, sheet_name= 'Description')
print(descrip['Description'][0])

49 Industry Portfolios
----------------------

This file was created by CMPT_IND_RETS using the 202312 CRSP database. It contains value- and equal-weighted returns for 49 industry portfolios. The portfolios are constructed at the end of June. The annual returns are from January to December. Missing data are indicated by -99.99 or -999. Copyright 2023 Kenneth R. French

  0 : Average Value Weighted Returns -- Monthly (1170 rows x 49 cols)
  1 : Average Equal Weighted Returns -- Monthly (1170 rows x 49 cols)
  2 : Average Value Weighted Returns -- Annual (97 rows x 49 cols)
  3 : Average Equal Weighted Returns -- Annual (97 rows x 49 cols)
  4 : Number of Firms in Portfolios (1170 rows x 49 cols)
  5 : Average Firm Size (1170 rows x 49 cols)
  6 : Sum of BE / Sum of ME (98 rows x 49 cols)
  7 : Value-Weighted Average of BE/ME (98 rows x 49 cols)


In [11]:
op_inv_keys = list(op_inv_ports.keys())
filepath = f'../data/famafrench/{op_inv_keys[0]}.xlsx'
descrip = pd.read_excel(filepath, sheet_name= 'Description')
print(descrip['Description'][0])

This file was created by CMPT_ME_BEME_OP_INV_RETS using the 202312 CRSP database.
It contains value- and equal-weighted returns for portfolios formed on OP and INV.

The portfolios are constructed at the end of June. OP is operating profits (sales - cost
of goods sold - selling, general and administrative expenses - interest expense) divided by book
at the last fiscal year end of the prior calendar year. INV is the change
in total assets from the fiscal year ending in year t-2 to the fiscal year ending in t-1,
divided by t-2 total assets.
Annual returns are from January to December.

Missing data are indicated by -99.99 or -999.

Please be aware that some of the value-weight averages of operating profitability for
deciles 1 and 10 are extreme. These are driven by extraordinary values of OP for
individual firms. We have spot checked the accounting data that produce the
extraordinary values and all the numbers we examined accurately reflect the
data in the firms accounting statements.

